In [30]:
import pandas as pd
import numpy as np
ruta = "c:/Users/CO-149/Downloads/ResultadosPuestoValores_NOCVal.xlsx"
hv = pd.read_excel(ruta)
columnas = hv.columns
real_names = []
for x in columnas:
    real_names.append(hv[x][4])
real_names[0] = "no_name"
for z in range(0, 5):
    hv.drop(index=z, inplace=True)
hv.columns = real_names
hv.drop("no_name", axis=1, inplace=True)
    

In [33]:
import numpy as np
import pandas as pd
from datetime import datetime
import seaborn as sns
import urllib3
import json

hv = pd.DataFrame(data=None)


class SmartSearch:
    def __init__(self):
        pass

    @staticmethod
    def process_database(ruta="C:/Users/CO-149//Downloads/Hojas de Vida Creadas (2).csv",
                         ruta_out="c:/Users/CO-149/Downloads/out.csv"):
        hv = pd.read_csv(ruta)

        def normalize(s, replacements):
            for a, b in replacements:
                s = s.replace(a, b).replace(a.lower(), b.lower()).lower()
            return s

        column_names = []
        for x in hv.columns:
            y = normalize(x, (("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"), (" ", "_"), ("ñ", "n"), ("(", ""),
                (")", ""), (",", ""), (".", "")))
            column_names.append(y)

        hv.columns = column_names

        def predecir_genero(first_name):
            tablaTemporal = hv[(hv["primer_nombre"] == first_name) & (hv["genero_de_nacimiento"] != "NaN")]
            count = 0
            http = urllib3.PoolManager()
            for row in tablaTemporal.index:
                if tablaTemporal["genero_de_nacimiento"][row] == "MASCULINO":
                    count += 1
                else:
                    count -= 1
            if count == 0:
                url = "https://api.genderize.io/?name=" + first_name
                gender = json.loads(http.request('GET', url).data.decode('utf-8')).get("gender")
                if gender == "female":
                    response = "FEMENINO"
                elif gender == "male":
                    response = "MASCULINO"
                else:
                    response = "NaN"
            elif count > 0:
                response = "MASCULINO"
            else:
                response = "FEMENINO"
            return response

        table = hv[hv.genero_de_nacimiento.isnull()]
        for x in table.index:
            hv["genero_de_nacimiento"][x] = predecir_genero(str(hv["primer_nombre"][x]))

        hv["genero_femenino"] = np.where(hv["genero_de_nacimiento"] == "FEMENINO", 1, 0)

        x = 0
        hv["num_cargos"] = 0
        for index in range(0, len(hv)):
            if type(hv["cargos"][index]) == str:
                hv["num_cargos"][index] = hv["cargos"][index].count("| ,") + 1
            else:
                hv["num_cargos"][index] = 0

        # Contabilizamos la catidad de trabajos que ha tenido hasta la fecha
        Experiencia = "cargos"
        var_ref = "num_cargos"
        hv[[Experiencia, var_ref]].describe()
        sns.histplot(hv["num_cargos"])

        def string_to_list(cargo_string) -> object:
            ''' Convierne en matriz la xp donde cada vector es un registro de xp '''
            try:
                cargos = cargo_string.split("|| ,")
                listaCargos = []
                for cargo in cargos:
                    listaCargos.append(cargo.split(" || "))
            except Exception:
                listaCargos = []
            return listaCargos

        hv["lista_cargos"] = ""
        for x in hv.index:
            if hv["num_cargos"][x] == 0:
                hv["lista_cargos"][x] = ""
            else:
                hv["lista_cargos"][x] = string_to_list(hv["cargos"][x])

        hv["lista_estudios"] = ""
        for x in hv.index:
            try:
                hv["lista_estudios"][x] = string_to_list(hv["historia_escolar"][x])
            except Exception as e:
                hv["lista_estudios"][x] = ""

        def tranfor_to_date(date):
            ''' Convierto la data en fecha y reordeno los datos del año '''
            dateList = date.split("/")
            day = int(dateList[0])
            mount = int(dateList[1])
            year_str = dateList[2].strip()
            if (year_str[:2] != "20") | (len(year_str) != 4):
                if year_str[2] != ('0', '1', '2'):
                    new_year = "20" + year_str[2:]
                else:
                    new_year = "20" + year_str[3] + year_str[2]
                year = int(new_year)
            else:
                year = int(year_str)
            return datetime(year, mount, day)

        def calcular_experiencia(cargo_list, position, imprimir=False):
            # Devuelve la xp en años
            try:
                xp = 0
                for dataCargo in cargo_list:
                    if dataCargo[2] != '':
                        if dataCargo[3] == '':
                            xp_days = abs((hv["added_time"][position] - tranfor_to_date(dataCargo[2])).days)
                        else:
                            xp_days = abs((tranfor_to_date(dataCargo[3]) - tranfor_to_date(dataCargo[2])).days)
                    else:
                        xp_days = 0
                    xp = xp + xp_days  # type: ignore
                totalExperiencia = xp / 365
            except Exception:
                totalExperiencia = -1
            return totalExperiencia

        def info_experience(cargo_list, position=6, imprimir=False):
            ''' Concatena todos los textos en un solo bloque, elimina caracteres especiales'''
            funciones = ""
            try:
                for cargo in cargo_list:
                    if cargo[6] != "":
                        funciones = funciones + " " + normalize(cargo[position], (
                        ("á", "a"), ("é", "e"), ("í", "i"), ("ó", "o"), ("ú", "u"), (" ", " "), ("(", ""), (")", ""),
                        (",", ""), (".", ""), (";", ""), (":", ""), ("  ", " ")))
            except Exception:
                funciones = ""

            return funciones

        hv["xp"] = 0
        hv["info_candidato"] = ""
        for x in range(0, len(hv)):
            if hv["num_cargos"][x] > 0:
                hv["info_candidato"][x] = info_experience(hv["lista_cargos"][x]) + " " + info_experience(
                    hv["lista_estudios"][x], position=3)
                hv["xp"][x] = calcular_experiencia(hv["lista_cargos"][x], x)
            else:
                hv["xp"][x] = 0

        hv["lista_palabras"] = ""
        for x in hv.index:
            try:
                hv["lista_palabras"][x] = hv["info_candidato"][x].split(" ")
            except Exception as e:
                hv["lista_palabras"][x] = ""

        for x in hv.index:
            unique_sweets = []
            [unique_sweets.append(sweet) for sweet in hv["lista_palabras"][x] if sweet not in unique_sweets]
            hv["lista_palabras"][x] = unique_sweets

        hv["num_nivel_educativo"] = np.where(
            (hv["nivel_educativo"] == "NINGUNO"),
            0,
            np.where(
                (hv["nivel_educativo"] == "BASICA PRIMARIA"),
                1,
                np.where(
                    (hv["nivel_educativo"] == "MEDIA ACADEMICA (BACHILLER)"),
                    2,
                    np.where(
                        (hv["nivel_educativo"] == "TECNICA PROFESIONAL"),
                        3,
                        np.where(
                            (hv["nivel_educativo"] == "TECNOLOGICA"),
                            4, np.where(
                                (hv["nivel_educativo"] == "PROFESIONAL"),
                                5,
                                np.where(
                                    (hv["nivel_educativo"] == "POSTGRADO"),
                                    6,
                                    0
                                )
                            )
                        )
                    )
                )
            )
        )

        # Tiempo que una persona dura en promedio por cada cargo
        hv["estabilidad_laboral"] = 0
        for x in hv.index:
            try:
                hv["estabilidad_laboral"][x] = (hv["xp"][x] / hv["num_cargos"][x])
            except Exception as e:
                hv["estabilidad_laboral"][x] = 0
            if hv["estabilidad_laboral"][x] < 0:
                hv["estabilidad_laboral"][x] = 0

        hv["estabilidad_laboral"] = np.where(
            hv["estabilidad_laboral"].isnull(),
            0,
            np.where
                (
                hv["estabilidad_laboral"] > 20,
                20,
                hv["estabilidad_laboral"]
            )
        )

        hv.to_csv(path_or_buf=ruta_out)

    @staticmethod
    def smart_search(age_minima=18, age_maxima=54, gender=2, palabras_clave=None, nivel_educativo=0,
                     tiempo_experiencia=0, ruta="c:/Users/CO-149/Downloads/out.csv"):
        """ Función que busca y puntua cada uno de los criterios de busqueda especificados """
        if palabras_clave is None:
            palabras_clave = []
        base = pd.read_csv(ruta)
        lista_puntaje = []
        for x in base.index:
            score = 0

            """Genero 0.0625"""
            if gender != 2:
                if base["genero_femenino"][x] == gender:
                    score = score + 0.0625
            else:
                score = score + 0.0625

            """rango de edad  0.0625 """
            try:
                edad = int(base["edad"][x])  # type: ignore
                if edad >= age_minima & edad <= age_maxima:
                    score = score + 0.0625
            except Exception:
                score = score + 0

            """nivel educativo 0.125"""
            niv_educativo = base["num_nivel_educativo"][x]
            if niv_educativo == nivel_educativo:
                score = score + 0.125
            elif niv_educativo >= nivel_educativo:
                score = score + 0.0625

            """Experiencia 0.125"""
            experiencia = base["experiencia"][x]
            if experiencia >= tiempo_experiencia:
                score = score + 0.125

            """Match etiquetas 0.5"""
            lista_match = []
            num_palabras = len(palabras_clave)
            if num_palabras != 0:
                texto = base["lista_palabras"][x]
                for y in palabras_clave:
                    if texto.__contains__(y):
                        lista_match.append(y)
                score = score + ((len(lista_match) / num_palabras) * 0.5)
            else:
                score = score + 0.5

            lista_puntaje.append([score, lista_match])

        df = pd.DataFrame(lista_puntaje, columns=["score", "lista_match"])
        hv_final = pd.concat([base, df], axis=1)
        return hv_final


if __name__ == '__main__':
    start_time = datetime.now()
    objeto = SmartSearch()
    dataframe = SmartSearch.smart_search(gender=0,
                                         palabras_clave=["auxiliar", "bodega", "descargue"],
                                         nivel_educativo=0)
    display(dataframe)
    dataframe.to_csv("c:/Users/CO-149/Downloads/score.csv")
    deltaTime = datetime.now() - start_time
    print(deltaTime)

#


C:\Users\CO-149\AppData\Local\Temp\ipykernel_6680\1111786049.py:230: DtypeWarning: Columns (6,13,18,19,27,28,32,34,35,36,37,38,39,40,43,64,73,76,78,80,82,86,87,92,93,95,96,97,98,99,100,101,102,104,105,109,110,111,112,113,117,119,120,121,122,127,128,129,131,133) have mixed types. Specify dtype option on import or set low_memory=False.
  base = pd.read_csv(ruta)


,Unnamed: 0,primer_apellido,segundo_apellido,primer_nombre,segundo_nombre,tipo_de_documento,no_identificacion,nacionalidad,no_pasaporte,fecha_de_nacimiento,...,num_cargos,lista_cargos,lista_estudios,experiencia,info_candidato,lista_palabras,num_nivel_educativo,estabilidad_laboral,score,lista_match
0,0,ESPITIA,MUNZA,SHARON,CAMILO,CEDULA DE CIUDADANIA,1076670237,Colombia,NaN,15/07/1999,...,0,NaN,[],0.000000,NaN,[''],0,0.000000,0.312500,[]
1,1,ORTEGA,HERNANDEZ,EDITH,DELFINA,CEDULA DE CIUDADANIA,1033724167,Colombia,NaN,05/07/1990,...,0,NaN,[],0.000000,NaN,[''],0,0.000000,0.312500,[]
2,2,SARAVIA,NaN,WILSON,ANTONIO,CEDULA DE CIUDADANIA,88239301,Colombia,NaN,21/09/1979,...,5,"[['MEMPHIS PRODUCTS S.A', 'AUXILIAR D EPLANTA ...","[['MEDIA ACADEMICA (BACHILLER)', '01/01/2000',...",6.347945,brindar servicio de seguridad a claro colombi...,"['', 'brindar', 'servicio', 'de', 'seguridad',...",2,1.269589,0.479167,[auxiliar]
3,3,CUEVAS,CAMACHO,YEISON,ESTEBEN,CEDULA DE CIUDADANIA,1070988591,Colombia,NaN,22/05/1999,...,0,NaN,[],0.000000,NaN,[''],2,0.000000,0.312500,[]
4,4,BARRANTES,CICUA,CLAUDIA,CONSTANZA,CEDULA DE CIUDADANIA,52425780,Colombia,NaN,19/10/1977,...,4,"[['ORGANIZACION SERVICIOS Y ASESORIAS', 'SUPER...","[['TECNOLOGICA', '01/07/2018', '01/06/2019', '...",-1.000000,todo el proceso de seguimiento a requisicione...,"['', 'todo', 'el', 'proceso', 'de', 'seguimien...",4,0.000000,0.125000,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75967,75967,GARZON,VARGAS,YULIANA,ANDREA,CEDULA DE CIUDADANIA,37292765,Colombia,NaN,27/03/1988,...,1,"[['NEGOCIO FAMILIAR ', 'VENDEDOR DE MOSTRADOR ...","[['MEDIA TECNICA', '11/02/1998', '17/11/2003',...",9.241096,atencion al cliente despachos y manejo de mer...,"['', 'atencion', 'al', 'cliente', 'despachos',...",3,9.241096,0.250000,[]
75968,75968,TEST,GUZMAN,TEST,ANDRES,CEDULA DE CIUDADANIA,22222,Colombia,NaN,08/11/1997,...,0,NaN,"[['PROFESIONAL', '01/01/2018', '', '', 'CURSAN...",0.000000,NaN,[''],4,0.000000,0.312500,[]
75969,75969,PEREZ,GUZMAN,JUAN,ANDRES,CEDULA DE CIUDADANIA,11111,Colombia,NaN,08/11/1997,...,0,NaN,"[['PROFESIONAL', '01/01/2018', '', '', 'CURSAN...",0.000000,NaN,[''],4,0.000000,0.312500,[]
75970,75970,DANIEL,FELIPE,GOMEZ,SUAREZ,CEDULA DE CIUDADANIA,100000001,Colombia,NaN,13/11/1997,...,2,"[['COOMPHIA', 'ANALISTA DE RECLUTAMIENTO', '08...","[['MEDIA ACADEMICA (BACHILLER)', '01/01/2009',...",2.906849,test test,"['', 'test']",0,1.453425,0.375000,[]


0:00:15.370655
